## Step 1: Get .nii file conversion for your data 

This can be easily done using Jimmy Shen's MatLab package for NifTi analysis (http://www.mathworks.com/matlabcentral/fileexchange/8797-tools-for-nifti-and-analyze-image). 

After downloading the zip file, extract and then add in the files to the folder that you want to convert. After adding that folder into your MatLab directory, simply type in the following two lines (modifying for the file names).

nii=load_nii(‘filename_with_hdr_extension’);
save_nii(nii, ‘desired_filename_for_niftifile.nii’);

Now you have converted to .nii successfully!

## Step 2: Histogram Equalization 

IMPORTANT: This code was very largely modified from Scikit's histogram equalization algorithm. All credit should go to them.

In [1]:
import os
PATH="/Users/albertlee/claritycontrol/code/scripts"
os.chdir(PATH)

In [2]:
import clearity as cl  # I wrote this module for easier operations on data
import matplotlib.pyplot as plt
import jgraph as ig
import clearity.resources as rs
import csv,gc  # garbage memory collection :)
import matplotlib
#import matplotlib.pyplot as plt
import numpy as np
from skimage import data, img_as_float
from skimage import exposure

BINS=32 # histogram bins
RANGE=(10.0,300.0)

matplotlib.rcParams['font.size'] = 8

In [3]:
def plot_img_and_hist(img, axes, bins=256):
    """Plot an image along with its histogram and cumulative histogram.

    """
    img = img_as_float(img)
    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display image
    ax_img.imshow(img, cmap=plt.cm.gray)
    ax_img.set_axis_off()
    ax_img.set_adjustable('box-forced')

    # Display histogram
    ax_hist.hist(img.ravel(), bins=bins, histtype='step', color='black')
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')
    ax_hist.set_xlim(0, 1)
    ax_hist.set_yticks([])

    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(img, bins)
    ax_cdf.plot(bins, img_cdf, 'r')
    ax_cdf.set_yticks([])

    return ax_img, ax_hist, ax_cdf

In [4]:
import nibabel as nb
im = nb.load('../data/raw/Fear187.nii')
im = im.get_data()
img = im[:,:,:,0]
print img

[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 ..., 
 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]


In [ ]:
# Equalization
img_eq = exposure.equalize_hist(im)
img_eqfinal = nb.Nifti1Image(img_eq, np.eye(4))

nb.save(img_eqfinal, "../data/raw/Fear187.nii")

## Step 3: Visualization

Now that the data has been transformed we can visualize the new data.

In [ ]:
c = cl.Clarity("Fear187")
c.loadImg().imgToPoints(threshold=0.02,sample=0.3).showHistogram(bins=256)

Let's compare the results to the pre-equalized histogram image data.

In [ ]:
import clarity as cl
import clarity.resources as rs

c = cl.Clarity("Fear187")
c.loadImg().imgToPoints(threshold=0.02,sample=0.3).showHistogram(bins=256)

In [ ]:
c.loadImg().imgToPoints(threshold=0.04,sample=0.5).savePoints()
c.loadPoints().show()